# Data: https://www.kaggle.com/rahulsathyajit/shark-tank-pitches

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import *

In [ ]:
df = pd.read_csv('shark_tank.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
int(df.deal[0])

In [ ]:
df.deal=(df.deal).astype(int)

In [ ]:
df['Multiple Entreprenuers']=(df['Multiple Entreprenuers']).astype(int)

In [ ]:
df.head()

In [ ]:
df.category.value_counts()

In [ ]:
for i in range(len(df)):
    df.location.iloc[i]=df.location[i].split(',')[1]

In [ ]:
df.head()

In [ ]:
sns.distplot(df.askedFor)

In [ ]:
df['website_length']=0
for i in range(len(df)):
    try:
        df['website_length'][i]=len(df.website[i])
    except:
        df['website_length'][i]=0

In [ ]:
sns.violinplot(df.website_length)

In [ ]:
pd.cut(df.website_length,bins=3).unique()

In [ ]:
df.website_length=pd.cut(df.website_length,bins=3,labels=['short','medium','long'])

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
df.columns

In [ ]:
i=1
j=0
colors=['red','blue','yellow']
plt.figure(figsize=(15,8))
for col in ['askedFor','valuation','exchangeForStake']:
    plt.subplot(2,2,i)
    sns.violinplot(df[col],color=colors[j])
    i=i+1
    j+=1

In [ ]:
def filter_outliers(dataframe,threshold):
    """
    Input a data frame and have all outliers filtered to a certain and custom threshold of standard deviations
    """
    dataframe = dataframe[(np.abs(stats.zscore(dataframe)) <= threshold).all(axis=1)]
    return dataframe

In [ ]:
df.shape

In [ ]:
df.website_length=df.website_length.astype(str)

In [ ]:
df.info()

In [ ]:
df_filter_num = filter_outliers(df.select_dtypes(include=[int,float]),2)

In [ ]:
i=1
j=0
colors=['red','blue','yellow']
plt.figure(figsize=(15,8))
for col in ['askedFor','valuation','exchangeForStake']:
    plt.subplot(2,2,i)
    sns.violinplot(df_filter_num[col],color=colors[j])
    i=i+1
    j+=1

In [ ]:
df_filter_num['asked_size']=pd.cut(df_filter_num.askedFor,bins=3,labels=['small','medium','large'])
df_filter_num['value_size']=pd.cut(df_filter_num.askedFor,bins=3,labels=['small','medium','large'])

In [ ]:
df_filter_cat = df[df.index.isin(df_filter_num.index)].select_dtypes(include='O')

In [ ]:
df_filter_cat.head()

In [ ]:
df_filter=pd.concat([df_filter_cat,df_filter_num],axis=1)

In [ ]:
print(df_filter.shape)
df_filter.head()

In [ ]:
df_filter.reset_index(inplace=True)

In [ ]:
df_filter.head()

In [ ]:
df_filter.drop('index',axis=1,inplace=True)

In [ ]:
df_filter.shape

In [ ]:
sns.distplot(df_filter.askedFor)

In [ ]:
df_filter.head()

In [ ]:
df_filter.website.fillna('none',inplace=True)

In [ ]:
for i in range(len(df_filter)):
    if df_filter.website[i]=='none':
        df_filter.website[i]=0
    else:
        df_filter.website[i]=1

In [ ]:
df_filter.head()

In [ ]:
df_filter.category.value_counts()

In [ ]:
pitch_cat_df=pd.get_dummies(df_filter.category)

In [ ]:
pitch_cat_df=pitch_cat_df.drop("Women's Shoes",axis=1)

In [ ]:
for col in pitch_cat_df.columns:
    df_filter[col]=pitch_cat_df[col]

In [ ]:
df_filter.head()

In [ ]:
df_filter.columns

In [ ]:
df_filter.head()

In [ ]:
df_filter['sharks']=None

In [ ]:
for i in range(len(df_filter)):
    df_filter['sharks'][i]=[]

In [ ]:
for i in range(len(df_filter)):
    df_filter['sharks'][i].append([df_filter.shark1[i],df_filter.shark2[i],df_filter.shark3[i],df_filter.shark4[i],df_filter.shark5[i]])

In [ ]:
df_filter.head()

In [ ]:
for i in range(len(df_filter)):
    df_filter.sharks[i]=df_filter.sharks[i][0]

In [ ]:
df_filter.head()

In [ ]:
df_filter.info()

In [ ]:
df_filter.sharks.astype(str).unique()

In [ ]:
sharks_dict={
    'group_1':['Barbara Corcoran', 'Robert Herjavec', "Kevin O'Leary", 'Daymond John', 'Kevin Harrington'],
    'group_2':['Barbara Corcoran', 'Robert Herjavec', "Kevin O'Leary", 'Daymond John', 'Mark Cuban'],
    'group_3':['Barbara Corcoran', 'Robert Herjavec', "Kevin O'Leary", 'Jeff Foxworthy', 'Daymond John'],
    'group_4':['Lori Greiner', 'Robert Herjavec', "Kevin O'Leary", 'Daymond John', 'Mark Cuban'],
    'group_5':['Lori Greiner', 'Barbara Corcoran', 'Robert Herjavec', "Kevin O\'Leary", 'Mark Cuban'],
    'group_6':['Lori Greiner', "Kevin O'Leary", 'Daymond John', 'Mark Cuban', 'John Paul DeJoria'],
    'group_7':['Lori Greiner', 'Steve Tisch', "Kevin O'Leary", 'Daymond John', 'Mark Cuban'],
    'group_8':['Lori Greiner', "Kevin O'Leary", 'Daymond John', 'Mark Cuban', 'Nick Woodman']
}

In [ ]:
df_filter['shark_group']=None

In [ ]:
for i in range(len(df_filter)):
    if df_filter.sharks[i]==sharks_dict['group_1']:
        df_filter.shark_group[i]=1
    elif df_filter.sharks[i]==sharks_dict['group_2']:
        df_filter.shark_group[i]=2
    elif df_filter.sharks[i]==sharks_dict['group_3']:
        df_filter.shark_group[i]=3
    elif df_filter.sharks[i]==sharks_dict['group_4']:
        df_filter.shark_group[i]=4
    elif df_filter.sharks[i]==sharks_dict['group_5']:
        df_filter.shark_group[i]=5
    elif df_filter.sharks[i]==sharks_dict['group_6']:
        df_filter.shark_group[i]=6
    elif df_filter.sharks[i]==sharks_dict['group_7']:
        df_filter.shark_group[i]=7
    else:
        df_filter.shark_group[i]=8

In [ ]:
df_filter.columns

In [ ]:
df_filter.iloc[:,0:15].head()

In [ ]:
df_filter.iloc[:,15:30].head()

In [ ]:
df_filter.iloc[:,30:45].head()

In [ ]:
df_filter.iloc[:,45:].head()

In [ ]:
df_filter.columns

In [ ]:
df_filter=df_filter[['deal','description','category', 'entrepreneurs', 'location', 'website', 'website_length','shark1',
       'shark2', 'shark3', 'shark4', 'shark5','sharks',
       'shark_group' ,'title', 'episode-season', 'episode', 'askedFor', 'exchangeForStake', 'valuation',
       'season', 'Multiple_Entreprenuers', 'asked_size', 'value_size',
       'Alcoholic Beverages', 'Automotive', 'Baby and Child Care',
       "Baby and Children's Apparel and Accessories",
       "Baby and Children's Bedding", "Baby and Children's Entertainment",
       "Baby and Children's Food", 'Consumer Services', 'Costumes', 'Cycling',
       'Education', 'Electronics', 'Entertainment', 'Fashion Accessories',
       'Fitness Apparel and Accessories', 'Fitness Equipment',
       'Fitness Programs', 'Furniture', 'Gardening', 'Golf Products',
       'Health and Well-Being', 'Holiday Cheer', 'Home Accessories',
       'Home Improvement', 'Home Security Solutions', 'Homeopathic Remedies',
       'Kitchen Tools', 'Maternity', "Men and Women's Accessories",
       "Men and Women's Apparel", "Men and Women's Shoes", "Men's Accessories",
       'Mobile Apps', 'Music', 'Non-Alcoholic Beverages', 'Novelties',
       'Online Services', 'Outdoor Recreation', 'Party Supplies',
       'Personal Care and Cosmetics', 'Pest Control', 'Pet Products',
       'Productivity Tools', 'Professional Services', 'Specialty Food',
       'Storage and Cleaning Products', 'Toys and Games',
       'Undergarments and Basics', 'Water Bottles', 'Weddings',
       'Wine Accessories', "Women's Accessories", "Women's Apparel"]]

In [ ]:
df_filter.head()

In [ ]:
df_filter_basic=df_filter.iloc[:,0:24]
df_filter_dummies=df_filter.iloc[:,24:]

In [ ]:
#df_filter_basic.to_csv('cleaned_shark.csv')

# I need to address categorical data numerically

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(abs(df_filter_basic.corr()>=0.7),cmap='plasma')
plt.tight_layout()
plt.title('correlation between features matchin 70% or higher')
plt.show()

In [ ]:
df_filter_basic.info()

In [ ]:
def target_encode(df,target,col):
    
    dummy_dict = {}
    dummy_df = df[[col,target]].groupby(col,as_index=False).mean()
    for i in range(len(dummy_df)):
        dummy_dict[dummy_df.iloc[i,0]]=dummy_df.iloc[i,1]
    df[col]=df[col].map(lambda x: dummy_dict[x])
    
    return df[col]

In [ ]:
df_filter_basic.columns

In [ ]:
# drop unique and un-needed values
df_filter_basic.drop(['description','sharks','title','episode-season','category','entrepreneurs'],axis=1,inplace=True)

In [ ]:
df_filter_basic.info()

In [ ]:
df_filter_basic.shark_group=df_filter_basic.shark_group.astype(float)

In [ ]:
df_filter_basic.info()

In [ ]:
for col in df_filter_basic.select_dtypes(exclude=[int,float]).columns:
    df_filter_basic[col]=target_encode(df_filter_basic,'deal',col)

In [ ]:
df_filter_basic.info()

In [ ]:
for col in df_filter_basic.columns:
    df_filter_basic[col]=df_filter_basic[col].astype(float)

In [ ]:
df_filter_basic.info()

In [ ]:
for col in df_filter_dummies.columns:
    df_filter_dummies[col]=df_filter_dummies[col].astype(float)

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(abs(df_filter_basic.corr()>=0.7),cmap='mako')
plt.tight_layout()
plt.title('correlation between features matchin 70% or higher')
plt.show()

In [ ]:
df_filter_basic.drop(['season','value_size'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(abs(df_filter_basic.corr()>=0.8),cmap='magma_r')
plt.tight_layout()
plt.title('correlation between features matchin 80% or higher')
plt.show()

In [ ]:
df_filter_basic.drop(['shark_group'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(abs(df_filter_basic.corr()>=0.85),cmap='summer')
plt.tight_layout()
plt.title('correlation between features matchin 85% or higher')
plt.show()

# I'd like to keep info on which shark was in even though it has high correlation

In [ ]:
df = pd.concat([df_filter_basic,df_filter_dummies],axis=1)

In [ ]:
df.head()

In [ ]:
df['Target']=df.deal

In [ ]:
y=df.Target
X=df.drop(['deal','Target'],axis=1)

In [ ]:
# scale data
X_ss=df.copy()
ss = StandardScaler()
for col in X_ss.columns:
    X_ss[col]=scaler.fit_transform(X_ss[[col]])

In [ ]:
# scale data
X_ma=X.copy()
X_ma = MaxAbsScaler()
for col in df_ma.columns:
    X_ma[col]=scaler.fit_transform(X_ma[[col]])

In [ ]:
# dimensionality reduction
## this code is designed to find the number of features needed in the X data for an optimal accuracy score

def optimize_score_rfe(scaler, dataframe,method,style,target_variable,ts,cross_val=5,goal='accuracy'):
    
    """
    Input a goal, method, style (the method of analysis written as a string), and number of cross validations and 
    receive an optimal score, number of features used, and list of features used as an output
    """
    
    # Imports and initializing variables
    mms = scaler
    old_score = 0
    old_features_used = 1

    # Optimize features needed
    for i in range(1,df.shape[1]):
        y = dataframe.Target
        X = dataframe.drop('Target',axis=1)
        selector = RFE(method,n_features_to_select=i)
        # selector = RFECV(estimator=method, step=1, cv=StratifiedKFold(cross_val), scoring=goal,min_features_to_select=i)
        selector = selector.fit(X, y.values.ravel())
        selected_columns = X.columns[selector.support_]
        new_y = dataframe.Target
        new_X = dataframe[selected_columns]
        for col in new_X.columns:
            new_X[col] = mms.fit_transform(new_X[[col]])
        X_train, X_test, y_train, y_test = train_test_split(new_X,new_y,test_size=ts,random_state=14) 
        method.fit(X_train,y_train)
        Y_pred = method.predict(X_test)
        if style == 'Linear_Regression':
            accuracy = r2_score(y_test, Y_pred)
        else:
            accuracy = accuracy_score(y_test, Y_pred)
        new_score = accuracy
        if new_score > old_score:
            score = new_score
            features_used = i
            old_score = new_score
            old_features_used = i
        else:
            score = old_score
            features_used = old_features_used
    
        # crate new data frame based on work
        # selector_new = RFECV(estimator=method, step=1, cv=StratifiedKFold(cross_val), scoring='accuracy',min_features_to_select=features_used)
        selector_new = RFE(method,n_features_to_select=features_used)
        selector_new = selector_new.fit(X, y.values.ravel())
        selected_columns_new = X.columns[selector_new.support_]
        
    # print discoveries
    print(f'Predicting: {target_variable}')
    print('')
    print(f'The optimal score is {100*round(score,2)}%, and it makes use of {features_used} features from the data.')
    print('')
    print(f'Those features are: {list(selected_columns_new)}')

In [ ]:
# dimensionality reduction
## principal component analysis

def run_pca(X_tr,components):
    pca = PCA(n_components=components)
    pca.fit(X_tr)
    cumsum = np.cumsum(pca.explained_variance_ratio_)*100
    d = [n for n in range(len(cumsum))]
    plt.figure(figsize=(6,4))
    plt.plot(d, cumsum, color='red', label = 'Explained Variance')
    plt.title('Explained Variance vs Number of Components')
    plt.ylabel('Explained Variance')
    plt.xlabel('Number of Components')
    plt.axhline(y = 90, color='k', linestyle='--', label = '90% of Explained Variance')
    plt.legend(loc='best')
    #X_train_pca = pd.DataFrame(pca_fraud.fit_transform(X_train))
    #X_test_pca = pd.DataFrame(pca_fraud.transform(X_test))
    #train_pca = X_train_pca.copy()
    #train_pca.target=y_train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14,test_size=0.25)

In [ ]:
def classification_model(mod):
    model=mod
    model.fit(X_train,y_train)
    pred=mod.predict(X_test)
    sns.heatmap(confusion_matrix(y_test,pred),cmap='winter')
    print(classification_report(y_test,pred))

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

svm=SVC()
knn=KNeighborsClassifier()
logreg=LogisticRegression()
sgd=SGDClassifier()
rf=RandomForestClassifier(random_state=14)

In [ ]:
classification_model(rf)

In [ ]:
fi=pd.DataFrame(rf.feature_importances_).T

In [ ]:
fi.columns=X.columns

In [ ]:
fi_1=fi.T.sort_values(by=0,ascending=False)

In [ ]:
with sns.axes_style({'axes.facecolor':'black'}):
    fi_1[0:25].plot(kind='barh',figsize=(15,8),color='white')
    plt.tight_layout()
    plt.title('feature importances by magnitude')
    plt.show()

In [ ]:
logreg.fit(X_train,y_train)
pred=rf.predict(X_test)
sns.heatmap(confusion_matrix(y_test,pred),cmap='winter')
print(classification_report(y_test,pred))

In [ ]:
logreg.coef_

In [ ]:
fi=pd.DataFrame(logreg.coef_)

In [ ]:
fi.columns=X.columns

In [ ]:
fi_1=fi.T.sort_values(by=0,ascending=False)

In [ ]:
with sns.axes_style({'axes.facecolor':'m'}):
    fi_1.plot(kind='barh',figsize=(15,9),color='limegreen')
    plt.tight_layout()
    plt.title('feature importances with sign')
    plt.show()

In [ ]:
# run model again with no sharks correlation and with gridsearch

In [ ]:
df.drop(['shark1','shark2','shark3','shark4','shark5'],axis=1,inplace=True)

In [ ]:
y=df.Target
X=df.drop(['deal','Target'],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14,test_size=0.25)

In [ ]:
classification_model(logreg)

In [ ]:
fi=pd.DataFrame(logreg.coef_)

In [ ]:
fi.columns=X_train.columns

In [ ]:
fi_1=fi.T.sort_values(by=0,ascending=False)

In [ ]:
with sns.axes_style({'axes.facecolor':'lightgreen'}):
    fi_1.plot(kind='barh',figsize=(15,9),color='forestgreen')
    plt.tight_layout()
    plt.title('feature importances with sign')
    plt.show()

In [ ]:
# pipeline

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline